## Experiments: Sensitive Analysis Hyperparameter RSAST:


It is runned RSAST in a set of UCR datasets with a predefined number of runs ("runs"). Then, it is selected a range ("range_total") between 1, 10, 30 ,50 and 100 for the selected dataset.

In [105]:
import sys 
import os 
#add sast library path
file_path = os.path.dirname(os.getcwd())+"/sast"

#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.path.dirname(os.getcwd())+"\sast"


#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.getcwd()+"/sast"


#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.getcwd()+"\sast"


#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

#add cd_diagram library path
file_path = os.path.dirname(os.getcwd())+"\cd_diagram"

#file_path = r"C:\Users\Public\random_sast\cd_diagram"
sys.path.append(file_path)


file_path = os.path.dirname(os.getcwd())+"/cd_diagram"
#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.getcwd()+"\cd_diagram"
#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.getcwd()+"/cd_diagram"
#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

sys.path


['c:\\Users\\Nicolas R\\random_sast\\ExperimentationRSAST',
 'c:\\ProgramData\\Anaconda3\\python39.zip',
 'c:\\ProgramData\\Anaconda3\\DLLs',
 'c:\\ProgramData\\Anaconda3\\lib',
 'c:\\ProgramData\\Anaconda3',
 '',
 'C:\\Users\\Nicolas R\\AppData\\Roaming\\Python\\Python39\\site-packages',
 'c:\\ProgramData\\Anaconda3\\lib\\site-packages',
 'c:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32',
 'c:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32\\lib',
 'c:\\ProgramData\\Anaconda3\\lib\\site-packages\\Pythonwin',
 'c:\\ProgramData\\Anaconda3\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\Nicolas R\\.ipython',
 'c:\\Users\\Nicolas R\\random_sast/sast',
 'c:\\Users\\Nicolas R\\random_sast\\sast',
 'c:\\Users\\Nicolas R\\random_sast\\ExperimentationRSAST/sast',
 'c:\\Users\\Nicolas R\\random_sast\\ExperimentationRSAST\\sast',
 'c:\\Users\\Nicolas R\\random_sast\\cd_diagram',
 'c:\\Users\\Nicolas R\\random_sast/cd_diagram',
 'c:\\Users\\Nicolas R\\random_sast\\Experimentation

In [106]:
from sast import *
import pandas as pd
import researchpy
import math
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import seaborn as sns

### Reading Datasets

In [107]:
# Set directory where the csv files are located
directory = os.getcwd()+'/ResultsByClassifier'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        # Append the dataframe to the list
        df['filename']=filename
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_other_methods = pd.concat(dfs, ignore_index=True)
df_other_methods=df_other_methods[['filename','folds:','0']]
df_other_methods['method']=df_other_methods['filename'].str.split('_').str[0]
df_other_methods
df_other_methods=df_other_methods.rename(columns={'folds:':'dataset','0':'score'})
df_other_methods=df_other_methods[['dataset','score','method']]


In [108]:
df_other_methods['method'].unique()

array(['BOSS', 'Catch22', 'cBOSS', 'HIVE-COTEv1', 'InceptionTime',
       'ProximityForest', 'ResNet', 'RISE', 'ROCKET', 'S-BOSS', 'STC',
       'TS-CHIEF', 'TSF', 'WEASEL'], dtype=object)

In [109]:
# Set directory where the csv files are located
#directory = os.getcwd()+'/ResultsRsast'
#directory = os.getcwd()+'/results_accuracy_per_ds'

directories=[]
#directories.append(os.getcwd()+'/ResultsRsast/Server17_Comparison_RSAST')
#directories.append(os.getcwd()+'/ResultsRsast/Server17_Comparison_RSAST_All')
#directories.append(os.getcwd()+'/ResultsRsast/Server17_Comparison_RSAST_Extra_1')
#directories.append(os.getcwd()+'/ResultsRsast/Server17_Comparison_RSAST_Extra_2')
directories.append(os.getcwd()+'/ResultsRsast/Server17_Resampling_Comparison_RSAST')
#directories.append(os.getcwd()+'/ResultsRsast/Server17_not_used_sast')
#directories.append(os.getcwd()+'/ResultsRsast/Server16_Hyperparameter_Tunning')
#directories.append(os.getcwd()+'/ResultsRsast/Server17_Hyperparameter_Tunning')
#directories.append(os.getcwd()+'/results_accuracy_per_ds')
#directories.append(os.getcwd()+'/results_accuracy_per_ds_10000')


# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for directory in directories:
    for filename in os.listdir(directory):
        # Check if the file starts with "df_overall_tunning" and ends with ".csv"
        if filename.startswith("df_all_overall_tunning") and filename.endswith(".csv"):
            # Read the csv file into a dataframe
            filepath = os.path.join(directory, filename)
            df = pd.read_csv(filepath)
            df['rpoint']=df['rpoint'].astype(str)
            df['nb_per_class']=df['nb_per_class'].astype(str)
            df['rpoint']=df['rpoint'].replace("(lenthg ts)//2","half_len")
            df['nb_per_class']=df['nb_per_class'].replace("(max instances per class)//2","half_instance")
            df['classifier_name']=df['classifier_name'].str.replace("\(lenthg ts\)//2","half_len")
            df['classifier_name']=df['classifier_name'].str.replace("\(max instances per class\)//2","half_instance")
            # Append the dataframe to the list
            dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_result = pd.concat(dfs, ignore_index=True)
# df_result.head(3)


C:\Users\Nicolas R\AppData\Local\Temp\ipykernel_14448\94245683.py:33: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace("\(lenthg ts\)//2","half_len")
C:\Users\Nicolas R\AppData\Local\Temp\ipykernel_14448\94245683.py:34: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace("\(max instances per class\)//2","half_instance")
C:\Users\Nicolas R\AppData\Local\Temp\ipykernel_14448\94245683.py:33: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace("\(lenthg ts\)//2","half_len")
C:\Users\Nicolas R\AppData\Local\Temp\ipykernel_14448\94245683.py:34: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace(

In [110]:
df_used_sast=pd.read_excel("DataSetsUCLASummary.xlsx", dtype=str)
df_used_sast.columns

Index(['USED SAST', 'BAKE OFF', 'N RUNS S17_S16_HT', 'N RUNS S17_SAST_DS',
       'ID', 'Name', 'Type', 'Train ', 'Test ', 'Class', 'Length',
       'RSAST (10,10) WORST SAST?', 'O(|c|nm²)', 'O(|c|nm³)',
       'Approx Time (hours)', 'L25', 'L50', 'L75', 'ED (w=0)',
       'DTW (learned_w) ', 'DTW (w=100)', 'Default rate', 'Data donor/editor',
       '50/L', 'Unnamed: 24'],
      dtype='object')

In [111]:
df_used_sast=df_used_sast[['Name', 'Type', 'Train ', 'Test ', 'Class', 'Length','USED SAST']]
df_used_sast=df_used_sast[df_used_sast['USED SAST']=='Y']


In [112]:
print("Total DS:" + str(len(df_result['dataset_name'].unique())))

Total DS:42


In [60]:
pv_all=pd.pivot_table(df_result,index='dataset_name',values='accuracy',aggfunc="count")
ds_complete=pv_all[pv_all.accuracy==max(pv_all.accuracy)].reset_index().dataset_name.unique()
ds_complete
df_result=df_result[df_result.dataset_name.isin(ds_complete)]

In [61]:
pv_all=pd.pivot_table(df_result,index='len_method',values='accuracy',aggfunc="count")
pv_all

,accuracy
len_method,
ACF&PACF,1640
Max PACF,1640
None,1640


In [62]:
pv_all=pd.pivot_table(df_result,index='len_method',values='accuracy',aggfunc="count")
len_complete=pv_all[pv_all.accuracy==max(pv_all.accuracy)].reset_index().len_method.unique()
len_complete
df_result=df_result[df_result.len_method.isin(len_complete)]

In [63]:
df_result=df_result[df_result.nb_per_class.isin(['1', '10'])]

In [64]:
#get ds tested overall
print("Total DS:" + str(len(df_result['dataset_name'].unique())))

Total DS:41


In [65]:
#get ds tested overall
pd.pivot_table(df_result, index="dataset_name", values="accuracy", aggfunc="count").reset_index()["dataset_name"].unique()

array(['ACSF1', 'Adiac', 'AllGestureWiimoteX', 'AllGestureWiimoteY',
       'AllGestureWiimoteZ', 'ArrowHead', 'BME', 'Beef', 'BeetleFly',
       'BirdChicken', 'CBF', 'Car', 'Chinatown', 'ChlorineConcentration',
       'CinCECGTorso', 'Coffee', 'Computers', 'CricketX', 'CricketY',
       'CricketZ', 'Crop', 'DiatomSizeReduction',
       'DistalPhalanxOutlineAgeGroup', 'DistalPhalanxOutlineCorrect',
       'DistalPhalanxTW', 'DodgerLoopDay', 'DodgerLoopGame',
       'DodgerLoopWeekend', 'ECG200', 'ECG5000', 'ECGFiveDays',
       'EOGHorizontalSignal', 'Earthquakes', 'ElectricDevices', 'FaceAll',
       'FaceFour', 'FacesUCR', 'FiftyWords', 'Fish', 'FordA', 'FordB'],
      dtype=object)

In [66]:
#get ds tested overall
print("Total Param Combination:" + str(len(df_result['classifier_name'].unique())))

Total Param Combination:12


In [67]:
#get len methods of generated datasets
df_result.classifier_name.unique()

array(['ACF&PACF: n_random_points=10 nb_inst_per_class=1',
       'ACF&PACF: n_random_points=10 nb_inst_per_class=10',
       'ACF&PACF: n_random_points=30 nb_inst_per_class=1',
       'ACF&PACF: n_random_points=30 nb_inst_per_class=10',
       'Max PACF: n_random_points=10 nb_inst_per_class=1',
       'Max PACF: n_random_points=10 nb_inst_per_class=10',
       'Max PACF: n_random_points=30 nb_inst_per_class=1',
       'Max PACF: n_random_points=30 nb_inst_per_class=10',
       'None: n_random_points=10 nb_inst_per_class=1',
       'None: n_random_points=10 nb_inst_per_class=10',
       'None: n_random_points=30 nb_inst_per_class=1',
       'None: n_random_points=30 nb_inst_per_class=10'], dtype=object)

In [68]:
#get columns of generated datasets
df_result.columns.unique()

Index(['Unnamed: 0', 'accuracy', 'time', 'cweights_time', 'fsubsequence_time',
       'tdataset_time', 'tclassifier_time', 'dataset_name', 'classifier_name',
       'rpoint', 'nb_per_class', 'method', 'len_method'],
      dtype='object')

In [69]:
pd.pivot_table(df_result, index="dataset_name", values="accuracy", columns="classifier_name").reset_index()

classifier_name,dataset_name,ACF&PACF: n_random_points=10 nb_inst_per_class=1,ACF&PACF: n_random_points=10 nb_inst_per_class=10,ACF&PACF: n_random_points=30 nb_inst_per_class=1,ACF&PACF: n_random_points=30 nb_inst_per_class=10,Max PACF: n_random_points=10 nb_inst_per_class=1,Max PACF: n_random_points=10 nb_inst_per_class=10,Max PACF: n_random_points=30 nb_inst_per_class=1,Max PACF: n_random_points=30 nb_inst_per_class=10,None: n_random_points=10 nb_inst_per_class=1,None: n_random_points=10 nb_inst_per_class=10,None: n_random_points=30 nb_inst_per_class=1,None: n_random_points=30 nb_inst_per_class=10
0,ACSF1,0.339000,0.292000,0.306000,0.293000,0.412000,0.339000,0.384000,0.303000,0.445000,0.314000,0.418000,0.278000
1,Adiac,0.649361,0.625064,0.642199,0.602302,0.721995,0.539898,0.504859,0.562660,0.721483,0.648338,0.616880,0.661381
2,AllGestureWiimoteX,0.555429,0.669714,0.620857,0.675286,0.486286,0.525429,0.455429,0.600714,0.450429,0.478571,0.449857,0.550714
3,AllGestureWiimoteY,0.575286,0.709286,0.636143,0.715286,0.521286,0.516571,0.494571,0.655571,0.503000,0.491286,0.450286,0.555857
4,AllGestureWiimoteZ,0.467143,0.630857,0.561286,0.626143,0.446714,0.346143,0.445571,0.523571,0.427571,0.354286,0.402571,0.432286
5,ArrowHead,0.756571,0.780000,0.755429,0.777714,0.728000,0.731429,0.680000,0.745143,0.728571,0.704000,0.661143,0.716571
6,BME,0.929333,0.952667,0.927333,0.952667,0.786667,0.870000,0.788000,0.892667,0.750667,0.889333,0.757333,0.901333
7,Beef,0.466667,0.480000,0.430000,0.486667,0.336667,0.433333,0.313333,0.426667,0.326667,0.460000,0.363333,0.443333
8,BeetleFly,0.720000,0.850000,0.750000,0.850000,0.600000,0.830000,0.560000,0.820000,0.650000,0.710000,0.625000,0.745000
9,BirdChicken,0.740000,0.815000,0.725000,0.795000,0.690000,0.755000,0.650000,0.770000,0.660000,0.645000,0.750000,0.700000


In [70]:
"""
df_tunned_rsast=df_used_sast.merge(pd.pivot_table(df_result, index="dataset_name", values="accuracy", columns="classifier_name").reset_index(),how='right',right_on="dataset_name",left_on="Name").round(2)[['Name', 'ACF&PACF: n_random_points=30 nb_inst_per_class=10','ACF&PACF: n_random_points=30 nb_inst_per_class=10','ACF&PACF: n_random_points=30 nb_inst_per_class=1','Type', 'Train ', 'Test ', 'Class', 'Length']]
df_tunned_rsast.sort_values("Name",inplace=True)
df_tunned_rsast.to_excel('ds_runned_rsast.xlsx')
"""

'\ndf_tunned_rsast=df_used_sast.merge(pd.pivot_table(df_result, index="dataset_name", values="accuracy", columns="classifier_name").reset_index(),how=\'right\',right_on="dataset_name",left_on="Name").round(2)[[\'Name\', \'ACF&PACF: n_random_points=30 nb_inst_per_class=10\',\'ACF&PACF: n_random_points=30 nb_inst_per_class=10\',\'ACF&PACF: n_random_points=30 nb_inst_per_class=1\',\'Type\', \'Train \', \'Test \', \'Class\', \'Length\']]\ndf_tunned_rsast.sort_values("Name",inplace=True)\ndf_tunned_rsast.to_excel(\'ds_runned_rsast.xlsx\')\n'

### Hyperparameter Tunning RSAST 

#### Accuracy: Subsequence Lenght Method 

In [71]:
filter_param=df_result

#filter_param=filter_param[filter_param.len_method=='ACF&PACF']
filter_param.dataset_name.unique()

array(['ACSF1', 'Adiac', 'AllGestureWiimoteX', 'AllGestureWiimoteY',
       'AllGestureWiimoteZ', 'ArrowHead', 'Beef', 'BeetleFly',
       'BirdChicken', 'BME', 'Car', 'CBF', 'Chinatown',
       'ChlorineConcentration', 'CinCECGTorso', 'Coffee', 'Computers',
       'CricketX', 'CricketY', 'CricketZ', 'Crop', 'DiatomSizeReduction',
       'DistalPhalanxOutlineAgeGroup', 'DistalPhalanxOutlineCorrect',
       'DistalPhalanxTW', 'DodgerLoopDay', 'DodgerLoopGame',
       'DodgerLoopWeekend', 'Earthquakes', 'ECG200', 'ECG5000',
       'ECGFiveDays', 'ElectricDevices', 'EOGHorizontalSignal', 'FaceAll',
       'FaceFour', 'FacesUCR', 'FiftyWords', 'Fish', 'FordA', 'FordB'],
      dtype=object)

In [72]:
filter_param.rpoint.unique()

array(['10', '30'], dtype=object)

In [73]:
#filter_param=filter_param[filter_param.rpoint.isin(['1', '10', '30'])]

In [74]:
filter_param.nb_per_class.unique()

array(['1', '10'], dtype=object)

In [75]:
#filter_param=filter_param[filter_param.nb_per_class.isin(['1', '10', '30'])]

In [76]:
# create a pivot table with the mean of score by dataset
len_method_vs_ds=pd.pivot_table(filter_param, values='accuracy', index=['len_method'],columns=['dataset_name'], aggfunc='mean')
len_method_vs_ds=np.transpose(len_method_vs_ds)#[['ACF','PACF']]
len_method_vs_ds=len_method_vs_ds.reset_index()
df_rocket=df_other_methods[df_other_methods["method"]=="SAST"]
merged_df = len_method_vs_ds.merge(df_rocket,left_on='dataset_name', right_on='dataset',  how='left')
merged_df=merged_df.drop('dataset',axis=1)
merged_df


,dataset_name,ACF&PACF,Max PACF,None,score,method
0,ACSF1,0.307500,0.359500,0.363750,NaN,NaN
1,Adiac,0.629731,0.582353,0.662020,NaN,NaN
2,AllGestureWiimoteX,0.630321,0.516964,0.482393,NaN,NaN
3,AllGestureWiimoteY,0.659000,0.547000,0.500107,NaN,NaN
4,AllGestureWiimoteZ,0.571357,0.440500,0.404179,NaN,NaN
5,ArrowHead,0.767429,0.721143,0.702571,NaN,NaN
6,BME,0.940500,0.834333,0.824667,NaN,NaN
7,Beef,0.465833,0.377500,0.398333,NaN,NaN
8,BeetleFly,0.792500,0.702500,0.682500,NaN,NaN
9,BirdChicken,0.768750,0.716250,0.688750,NaN,NaN


In [77]:
merged_df.describe()

,ACF&PACF,Max PACF,None,score
count,41.000000,41.000000,41.000000,0.0
mean,0.783632,0.741658,0.736118,NaN
std,0.158161,0.170342,0.162736,NaN
min,0.307500,0.359500,0.363750,NaN
25%,0.706115,0.641700,0.662020,NaN
50%,0.782790,0.755116,0.750441,NaN
75%,0.921064,0.852962,0.875244,NaN
max,0.999716,0.990179,0.974107,NaN


In [78]:

# Summary statistics for a Series (single variable)

summ_ds=filter_param
summ_ds=researchpy.summary_cont(summ_ds.groupby(['dataset_name'])['accuracy'], conf = 0.95)
summ_ds

,N,Mean,SD,SE,95% Conf.,Interval
dataset_name,,,,,,
ACSF1,120,0.3436,0.0619,0.0057,0.3324,0.3548
Adiac,120,0.6247,0.0647,0.0059,0.6130,0.6364
AllGestureWiimoteX,120,0.5432,0.0830,0.0076,0.5282,0.5582
AllGestureWiimoteY,120,0.5687,0.0901,0.0082,0.5524,0.5850
AllGestureWiimoteZ,120,0.4720,0.0951,0.0087,0.4548,0.4892
ArrowHead,120,0.7304,0.0516,0.0047,0.7211,0.7397
BME,120,0.8665,0.0886,0.0081,0.8505,0.8825
Beef,120,0.4139,0.0873,0.0080,0.3981,0.4297
BeetleFly,120,0.7258,0.1145,0.0105,0.7051,0.7465


In [79]:
stats = filter_param.groupby(['len_method'])['accuracy'].agg(['mean', 'count', 'std'])

ci95_hi = []
ci95_lo = []

for i in stats.index:
    m, c, s = stats.loc[i]
    ci95_hi.append(m + 1.96*s/math.sqrt(c))
    ci95_lo.append(m - 1.96*s/math.sqrt(c))

stats['ci95_hi'] = ci95_hi
stats['ci95_lo'] = ci95_lo
print(stats.head(10))

                mean  count       std   ci95_hi   ci95_lo
len_method                                               
ACF&PACF    0.783632   1640  0.164015  0.791570  0.775694
Max PACF    0.741658   1640  0.180546  0.750396  0.732920
None        0.736118   1640  0.176531  0.744662  0.727574


#### Generate Boxplot Tunning Hyperparameter

In [80]:
df_result.nb_per_class.unique()

array(['1', '10'], dtype=object)

In [81]:
df_result.len_method.unique()

array(['ACF&PACF', 'Max PACF', 'None'], dtype=object)

In [82]:
#generate hyperparameter tuning boxplots

for k, ints in enumerate(df_result.nb_per_class.unique()):

    for len_m in df_result.len_method.unique():
        print("nb_per_class: "+str(ints))
        print("len_m: "+str(len_m))
        df_to_cd=df_result[df_result.classifier_name.str.contains(r'^'+len_m+':.*nb_inst_per_class='+str(ints)+'$')]
        #df_to_cd=pd.pivot_table(df_to_cd, index=['dataset_name','classifier_name'], columns=['rpoint'],values='accuracy')
        #df_to_cd = df_to_cd.reindex(columns=["1","10","30","50","100","half_len"])
        
        #print(df_to_cd.head(5))

        # Plot
        fig, ax = plt.subplots()
        
        
        #order=list(df_to_cd)
        sns.boxplot(data=df_to_cd, x='rpoint', y='accuracy', palette="Blues")
        #plt.boxplot(df_to_cd, labels=list(df_to_cd), showfliers=False)
        
        max_bx=max(df_to_cd.accuracy)
        min_bx=min(df_to_cd.accuracy)
        
        ax.set_ylim(min_bx,max_bx)
        # Axis details
        ax.set(xlabel='number of random points', ylabel='accuracy', title='Accuracy Boxplot for Len method:'+len_m+' and N° of Instances:'+ints)
        #plt.xticks([1, 2, 3, 4, 5, 6],list(df_to_cd) )
        #print(df_to_cd.describe())

        # save plot
        plt.savefig('images_boxplot_acc/boxplot_acc'+len_m+ints+'.png')




nb_per_class: 1
len_m: ACF&PACF
nb_per_class: 1
len_m: Max PACF
nb_per_class: 1
len_m: None
nb_per_class: 10
len_m: ACF&PACF
nb_per_class: 10
len_m: Max PACF
nb_per_class: 10
len_m: None


In [83]:
df_result.classifier_name.unique()

array(['ACF&PACF: n_random_points=10 nb_inst_per_class=1',
       'ACF&PACF: n_random_points=10 nb_inst_per_class=10',
       'ACF&PACF: n_random_points=30 nb_inst_per_class=1',
       'ACF&PACF: n_random_points=30 nb_inst_per_class=10',
       'Max PACF: n_random_points=10 nb_inst_per_class=1',
       'Max PACF: n_random_points=10 nb_inst_per_class=10',
       'Max PACF: n_random_points=30 nb_inst_per_class=1',
       'Max PACF: n_random_points=30 nb_inst_per_class=10',
       'None: n_random_points=10 nb_inst_per_class=1',
       'None: n_random_points=10 nb_inst_per_class=10',
       'None: n_random_points=30 nb_inst_per_class=1',
       'None: n_random_points=30 nb_inst_per_class=10'], dtype=object)

In [84]:
#generate hyperparameter tuning boxplots
for ds in df_result.dataset_name.unique():
    for k, ints in enumerate(df_result.nb_per_class.unique()):
        for len_m in df_result.len_method.unique():
            print("ds:"+ds)
            print("nb_per_class:"+str(ints))
            print("len_m:"+len_m)

            df_to_cd=df_result[df_result.classifier_name.str.contains(r'^'+len_m+':')]
            df_to_cd=df_to_cd[df_to_cd.dataset_name==ds]

            df_to_cd["time"]=df_to_cd['cweights_time']+df_to_cd['fsubsequence_time']+df_to_cd['tdataset_time']
            df_to_cd["time"]=df_to_cd["time"]/60
            max_bx=max(df_to_cd.accuracy)
            min_bx=min(df_to_cd.accuracy)
            
            df_to_cd=df_to_cd[df_to_cd.classifier_name.str.contains(r':.*nb_inst_per_class='+str(ints)+'$')]    

            #df_to_cd=pd.pivot_table(df_to_cd, index=['dataset_name','classifier_name'], columns=['rpoint'],values='accuracy')
            #df_to_cd = df_to_cd.reindex(columns=["1","10","30","50","100","half_len"])
            
            #print(df_to_cd.head(5))

            # Plot
            fig, ax = plt.subplots()
            
            
            #order=list(df_to_cd)
            sns.boxplot(data=df_to_cd, x='rpoint', y='accuracy', palette="Blues")
            #plt.boxplot(df_to_cd, labels=list(df_to_cd), showfliers=False)
            
            ax.set_ylim(min_bx,max_bx)
            # Axis details
            ax.set(xlabel='number of random points', ylabel='accuracy', title='Accuracy Boxplot for Len method:'+len_m+' and N° of Instances:'+ints)
            #plt.xticks([1, 2, 3, 4, 5, 6],list(df_to_cd) )
            #print(df_to_cd.describe())

            # save plot
            plt.savefig('images_boxplot_acc_per_ds/boxplot_acc'+len_m+ints+'_'+ds+'.png')

ds:ACSF1
nb_per_class:1
len_m:ACF&PACF
ds:ACSF1
nb_per_class:1
len_m:Max PACF
ds:ACSF1
nb_per_class:1
len_m:None


ds:ACSF1
nb_per_class:10
len_m:ACF&PACF
ds:ACSF1
nb_per_class:10
len_m:Max PACF
ds:ACSF1
nb_per_class:10
len_m:None
ds:Adiac
nb_per_class:1
len_m:ACF&PACF
ds:Adiac
nb_per_class:1
len_m:Max PACF
ds:Adiac
nb_per_class:1
len_m:None
ds:Adiac
nb_per_class:10
len_m:ACF&PACF
ds:Adiac
nb_per_class:10
len_m:Max PACF
ds:Adiac
nb_per_class:10
len_m:None
ds:AllGestureWiimoteX
nb_per_class:1
len_m:ACF&PACF
ds:AllGestureWiimoteX
nb_per_class:1
len_m:Max PACF
ds:AllGestureWiimoteX
nb_per_class:1
len_m:None
ds:AllGestureWiimoteX
nb_per_class:10
len_m:ACF&PACF
ds:AllGestureWiimoteX
nb_per_class:10
len_m:Max PACF
ds:AllGestureWiimoteX
nb_per_class:10
len_m:None
ds:AllGestureWiimoteY
nb_per_class:1
len_m:ACF&PACF
ds:AllGestureWiimoteY
nb_per_class:1
len_m:Max PACF
ds:AllGestureWiimoteY
nb_per_class:1
len_m:None
ds:AllGestureWiimoteY
nb_per_class:10
len_m:ACF&PACF
ds:AllGestureWiimoteY
nb_per_class:10
len_m:Max PACF
ds:AllGestureWiimoteY
nb_per_class:10
len_m:None
ds:AllGestureWiimoteZ
nb_per_class:1
len_

In [85]:
#generate hyperparameter tuning boxplots

for k, ints in enumerate(df_result.nb_per_class.unique()):
    for len_m in df_result.len_method.unique():
        
        
        df_to_cd=df_result[df_result.classifier_name.str.contains(r'^'+len_m+':.*nb_inst_per_class='+str(ints)+'$')]
        df_to_cd["time"]=df_to_cd['cweights_time']+df_to_cd['fsubsequence_time']+df_to_cd['tdataset_time']
        df_to_cd["time"]=df_to_cd["time"]/60
        #df_to_cd=pd.pivot_table(df_to_cd, index=['dataset_name','classifier_name'], columns=['rpoint'],values='accuracy')
        #df_to_cd = df_to_cd.reindex(columns=["1","10","30","50","100","half_len"])
        
        #print(df_to_cd.head(5))

        # Plot
        fig, ax = plt.subplots()
        
        
        #order=list(df_to_cd)
        sns.boxplot(data=df_to_cd, x='rpoint', y='time', palette="Blues")
        #plt.boxplot(df_to_cd, labels=list(df_to_cd), showfliers=False)
        
        max_bx=max(df_to_cd.time)
        min_bx=min(df_to_cd.time)
        ax.set_ylim(min_bx,max_bx)
        # Axis details
        ax.set(xlabel='number of random points', ylabel='time', title='Time Boxplot for Len method:'+len_m+' and N° of Instances:'+ints)
        #plt.xticks([1, 2, 3, 4, 5, 6],list(df_to_cd) )
        #print(df_to_cd.describe())

        # save plot
        plt.savefig('images_boxplot_time/boxplot_time'+len_m+ints+'.png')




C:\Users\Nicolas R\AppData\Local\Temp\ipykernel_14448\4020300904.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_cd["time"]=df_to_cd['cweights_time']+df_to_cd['fsubsequence_time']+df_to_cd['tdataset_time']
C:\Users\Nicolas R\AppData\Local\Temp\ipykernel_14448\4020300904.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_cd["time"]=df_to_cd["time"]/60
C:\Users\Nicolas R\AppData\Local\Temp\ipykernel_14448\4020300904.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

In [86]:
#generate hyperparameter tuning boxplots
for ds in df_result.dataset_name.unique():
    for k, ints in enumerate(df_result.nb_per_class.unique()):
        for len_m in df_result.len_method.unique():
        


            df_to_cd=df_result[df_result.classifier_name.str.contains(r'^'+len_m+':')]
            df_to_cd=df_to_cd[df_to_cd.dataset_name==ds]

            df_to_cd["time"]=df_to_cd['cweights_time']+df_to_cd['fsubsequence_time']+df_to_cd['tdataset_time']
            df_to_cd["time"]=df_to_cd["time"]/60
            max_bx=max(df_to_cd.time)
            min_bx=min(df_to_cd.time)

            df_to_cd=df_to_cd[df_to_cd.classifier_name.str.contains(r':.*nb_inst_per_class='+str(ints)+'$')]


            #df_to_cd=pd.pivot_table(df_to_cd, index=['dataset_name','classifier_name'], columns=['rpoint'],values='accuracy')
            #df_to_cd = df_to_cd.reindex(columns=["1","10","30","50","100","half_len"])
            
            #print(df_to_cd.head(5))

            # Plot
            fig, ax = plt.subplots()
            
            
            #order=list(df_to_cd)
            sns.boxplot(data=df_to_cd, x='rpoint', y='time', palette="Blues")
            #plt.boxplot(df_to_cd, labels=list(df_to_cd), showfliers=False)
            

            ax.set_ylim(min_bx,max_bx)
            # Axis details
            ax.set(xlabel='number of random points', ylabel='time', title='Time Boxplot for Len method:'+len_m+' and N° of Instances:'+ints)
            #plt.xticks([1, 2, 3, 4, 5, 6],list(df_to_cd) )
            #print(df_to_cd.describe())

            # save plot
            plt.savefig('images_boxplot_time_per_ds/boxplot_time'+len_m+ints+'_'+ds+'.png')


#### Generate CD Diagram Tunning Hyperparameter

In [87]:
df_perf=pd.pivot_table(df_result, values='accuracy', index=['classifier_name','dataset_name','len_method'], aggfunc='mean')
df_perf=df_perf.reset_index()
df_perf.classifier_name.unique()

array(['ACF&PACF: n_random_points=10 nb_inst_per_class=1',
       'ACF&PACF: n_random_points=10 nb_inst_per_class=10',
       'ACF&PACF: n_random_points=30 nb_inst_per_class=1',
       'ACF&PACF: n_random_points=30 nb_inst_per_class=10',
       'Max PACF: n_random_points=10 nb_inst_per_class=1',
       'Max PACF: n_random_points=10 nb_inst_per_class=10',
       'Max PACF: n_random_points=30 nb_inst_per_class=1',
       'Max PACF: n_random_points=30 nb_inst_per_class=10',
       'None: n_random_points=10 nb_inst_per_class=1',
       'None: n_random_points=10 nb_inst_per_class=10',
       'None: n_random_points=30 nb_inst_per_class=1',
       'None: n_random_points=30 nb_inst_per_class=10'], dtype=object)

In [88]:

df_perf[df_perf.classifier_name.str.contains(r'^'+len_m+':.*$')].classifier_name.unique()

array(['None: n_random_points=10 nb_inst_per_class=1',
       'None: n_random_points=10 nb_inst_per_class=10',
       'None: n_random_points=30 nb_inst_per_class=1',
       'None: n_random_points=30 nb_inst_per_class=10'], dtype=object)

In [89]:


from cd_function import *
#generate hyperparameter tuning cd diagrams

best_comb_by_method=[]
best=""
for len_m in df_perf.len_method.unique():
   df_to_cd=df_perf[df_perf.classifier_name.str.contains(r'^'+len_m+':.*$')]
   draw_cd_diagram(df_to_cd, labels=True, title=len_m +" comparison", fname='images_cd_diagram/cd-diagram_'+len_m+'.png')
   _, average_ranks, _ = wilcoxon_holm(df_perf=df_to_cd)
   min_rank= min(average_ranks)
   average_ranks=pd.DataFrame(average_ranks)
   best=average_ranks[average_ranks[0]==min_rank][0].index
   best_comb_by_method.append(best[0])



c:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
c:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


{0: 'ACF&PACF: n_random_points=10 nb_inst_per_class=1', 1: 'ACF&PACF: n_random_points=30 nb_inst_per_class=1', 2: 'ACF&PACF: n_random_points=10 nb_inst_per_class=10', 3: 'ACF&PACF: n_random_points=30 nb_inst_per_class=10'}
[0, 1]
[2, 3]
{0: 'Max PACF: n_random_points=10 nb_inst_per_class=1', 1: 'Max PACF: n_random_points=30 nb_inst_per_class=1', 2: 'Max PACF: n_random_points=10 nb_inst_per_class=10', 3: 'Max PACF: n_random_points=30 nb_inst_per_class=10'}
[0, 1]
[2, 3]
{0: 'None: n_random_points=10 nb_inst_per_class=1', 1: 'None: n_random_points=30 nb_inst_per_class=1', 2: 'None: n_random_points=10 nb_inst_per_class=10', 3: 'None: n_random_points=30 nb_inst_per_class=10'}
[0, 1]


c:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
c:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


In [90]:
best_comb_by_method

['ACF&PACF: n_random_points=10 nb_inst_per_class=10',
 'Max PACF: n_random_points=30 nb_inst_per_class=10',
 'None: n_random_points=30 nb_inst_per_class=10']

In [91]:
df_perf=pd.pivot_table(df_result, values='accuracy', index=['classifier_name','dataset_name','len_method'], aggfunc='mean')
df_perf=df_perf.reset_index()
df_to_cd=df_perf[df_perf.classifier_name.isin(best_comb_by_method)]

In [92]:
draw_cd_diagram(df_to_cd, labels=True, title="Best combination comparison", fname='images_cd_diagram/cd-diagram_best_com.png')


{0: 'Max PACF: n_random_points=30 nb_inst_per_class=10', 1: 'None: n_random_points=30 nb_inst_per_class=10', 2: 'ACF&PACF: n_random_points=10 nb_inst_per_class=10'}
[0, 1]


c:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


In [93]:
df_perf_all=df_perf[['dataset_name','len_method','accuracy']]
df_perf_all.rename(columns = {'len_method':'classifier_name'}, inplace = True)
df_perf_all=pd.pivot_table(df_perf_all, index=["dataset_name","classifier_name"]).reset_index()
df_perf_all

C:\Users\Nicolas R\AppData\Local\Temp\ipykernel_14448\797092933.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_perf_all.rename(columns = {'len_method':'classifier_name'}, inplace = True)


,dataset_name,classifier_name,accuracy
0,ACSF1,ACF&PACF,0.307500
1,ACSF1,Max PACF,0.359500
2,ACSF1,None,0.363750
3,Adiac,ACF&PACF,0.629731
4,Adiac,Max PACF,0.582353
...,...,...,...
118,FordA,Max PACF,0.777462
119,FordA,None,0.799242
120,FordB,ACF&PACF,0.821451
121,FordB,Max PACF,0.776852


In [94]:
draw_cd_diagram(df_perf_all, labels=True, fname='images_cd_diagram/cd-diagram_all_methods.png')


{0: 'None', 1: 'Max PACF', 2: 'ACF&PACF'}
[0, 1]


c:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


#### Focus on most accurate lenght method

##### Overall Accuracy

In [95]:
# highest accurate hyperparameters
best_comb_by_method

['ACF&PACF: n_random_points=10 nb_inst_per_class=10',
 'Max PACF: n_random_points=30 nb_inst_per_class=10',
 'None: n_random_points=30 nb_inst_per_class=10']

In [96]:
# filter by method with ACF&PACF
# create a pivot table with the mean of score by hyperparameter
df_result_acc=df_result
#df_result_acc=df_result_acc[df_result_acc["dataset_name"]=="Fungi"]

pivot = pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')
pivot = pivot.reindex(columns=["half_instance","1","10","30","50","100","1000","10000"], index=["half_len","1","10","30","50","100","1000","10000"])
pivot

nb_per_class,half_instance,1,10,30,50,100,1000,10000
rpoint,,,,,,,,
half_len,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,0.725877,0.777112,NaN,NaN,NaN,NaN,NaN
30,NaN,0.730907,0.781315,NaN,NaN,NaN,NaN,NaN
50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [97]:
# create a pivot table with the variatioon of score by hyperparameter
pivot = pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='var')
pivot = pivot.reindex(columns=["half_instance","1","10","30","50","100","1000","10000"], index=["half_len","1","10","30","50","100","1000","10000"])
pivot

nb_per_class,half_instance,1,10,30,50,100,1000,10000
rpoint,,,,,,,,
half_len,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,0.029759,0.030903,NaN,NaN,NaN,NaN,NaN
30,NaN,0.030493,0.028945,NaN,NaN,NaN,NaN,NaN
50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### Overall time complexity

In [98]:
df_result_acc.time.sum()/(60*60*24)

6.410759468489775

In [99]:
# create a pivot table with the mean of time spent in minutes
pivot=pd.pivot_table(df_result_acc, values=['time','cweights_time','fsubsequence_time','tdataset_time','tclassifier_time'], index=['dataset_name'], aggfunc='sum')/60
pivot = pivot.reindex(columns=['cweights_time','fsubsequence_time','tdataset_time','tclassifier_time','time'])
pivot

,cweights_time,fsubsequence_time,tdataset_time,tclassifier_time,time
dataset_name,,,,,
ACSF1,1.851993,3183.477921,73.873437,0.382206,3259.635894
Adiac,0.548829,12.748025,4.602362,6.234420,24.170215
AllGestureWiimoteX,0.661600,67.504561,4.264103,1.096137,73.537261
AllGestureWiimoteY,0.680451,64.542429,3.643381,0.932592,69.808239
AllGestureWiimoteZ,0.676557,65.363168,5.736469,0.921032,72.709707
ArrowHead,0.103809,2.943692,0.141755,0.313049,3.505838
BME,0.049957,0.444774,0.031507,0.007108,0.535001
Beef,0.246156,19.729545,0.682855,0.019052,20.687083
BeetleFly,0.164068,19.200939,0.220561,0.008184,19.598372


In [100]:
# create a pivot table with the mean of time spent in minutes by random points and instances per class
pivot = pd.pivot_table(df_result_acc, values='time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')/60
pivot = pivot.reindex(columns=["half_instance","1","10","30","50","100","1000","10000"], index=["half_len","1","10","30","50","100","1000","10000"])
pivot

nb_per_class,half_instance,1,10,30,50,100,1000,10000
rpoint,,,,,,,,
half_len,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,0.391862,3.335538,NaN,NaN,NaN,NaN,NaN
30,NaN,0.414560,3.363319,NaN,NaN,NaN,NaN,NaN
50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### Calculate weights time complexity

In [101]:
pd.pivot_table(df_result_acc, values='cweights_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10
rpoint,,
10,0.273391,0.240783
30,0.262301,0.248656


##### Finding subsequences time complexity

In [102]:
pd.pivot_table(df_result_acc, values='fsubsequence_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10
rpoint,,
10,20.358943,190.548314
30,20.517855,185.833116


##### Transform Dataset time complexity

In [103]:
pd.pivot_table(df_result_acc, values='tdataset_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10
rpoint,,
10,0.499392,3.961450
30,1.349213,11.581115


##### Classifier time complexity

In [104]:
pd.pivot_table(df_result_acc, values='tclassifier_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10
rpoint,,
10,2.373492,5.37769
30,2.740517,4.12595
